In [1]:
import warnings

warnings.filterwarnings("ignore", category=Warning)

import os
import pickle

from ydata_profiling import ProfileReport

from ml_assemblr.main_components.data_pod import DataPod
from ml_assemblr.main_components.column_type import ColumnType
from ml_assemblr.utils.string_case_utils import to_dummy

from kaggle_titanic.get_df import get_df
from kaggle_titanic.config import cfg
from kaggle_titanic.constant import *

In [2]:
from ml_assemblr.transformer.encoder.one_hot_encoder import OneHotEncoder
from ml_assemblr.transformer.column_type.features_setter import TopDownFeaturesSetter
from sklearn.model_selection import ShuffleSplit
from ml_assemblr.transformer.ml_common.cross_validator import CrossValidator
from ml_assemblr.transformer.column_type.column_type_setter import ColumnTypeSetter

In [3]:
dfs = get_df(cfg)

In [4]:
column_types = {MAIN: ColumnType(labels=[LABEL], splitters=[SPLIT], keys=[PRIMARY_KEY])}

dp = DataPod(dfs=dfs, main_df_name=MAIN, column_types=column_types, clean_column_name=to_dummy)

In [5]:
dp.peek_main_df()

1309 rows × 13 columns


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train


In [6]:
df_train = dp.slice_df(TRAIN)
df_prod = dp.slice_df(PRODUCTION)

In [7]:
# EDA reporting
for dataset_name, df in [(TRAIN, df_train), (PRODUCTION, df_prod)]:
    profile_report_path = f"{dataset_name}_profile_report.html"
    if os.path.exists(profile_report_path):
        continue
    profile = ProfileReport(df_train, title=f"Profiling Report for {dataset_name}", dark_mode=True)
    _ = profile.to_file(profile_report_path)

In [8]:
dp.peek_main_df()

1309 rows × 13 columns


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Split
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train


In [9]:
categorical_cols = ["Pclass", "Embarked"]

In [10]:
one_hot_encoder = OneHotEncoder(col_names=["Pclass", "Embarked"])
dp = dp.fit_transform(one_hot_encoder)

In [11]:
feature_setter = TopDownFeaturesSetter(excluded_col_names={"Name", "Ticket", "Cabin"})
dp = dp.fit_transform(feature_setter)

In [12]:
categorical_feature_setter = ColumnTypeSetter(column_type_map=dict(categorical_features=["Sex"]))
dp = dp.fit_transform(categorical_feature_setter)

In [13]:
cross_validator = CrossValidator(sklearn_cv=ShuffleSplit(n_splits=10, test_size=0.2, random_state=42))
dp = dp.fit_transform(cross_validator)

In [14]:
with open(cfg.research_cache_path / "01_dp.pkl", "wb") as f:
    pickle.dump(dp, f)